In [1]:
import numpy as np
import pandas as pd
import torch
from requests.exceptions import ChunkedEncodingError

import backend.config as config
import dataprocessing as dp
import rnn
import util

/Users/maximilianrapp/PycharmProjects/nextgenbike/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
raw_data = ""
while raw_data == "":
    try:
        raw_data = dp.get_data_db()
    except ChunkedEncodingError:
        print("Couldn't get Data, retrying ...")

dataframe = pd.read_json(raw_data)
print(dataframe.head())

Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
                         time  trip_id  vibration  latitude  longitude  \
0  2023-11-05 17:35:18.484000        0          0       0.0        0.0   
1  2023-11-05 17:35:18.537000        0          0       0.0        0.0   
2  2023-11-05 17:35:18.589000        0          0       0.0        0.0   
3  2023-11-05 17:35:18.641000        0          0       0.0        0.0   
4  2023-11-05 17:35:24.245000        0          0       0.0        0.0   

   acceleration_x  acceleration_y  acceleration_z  gyroscope_x  gyroscope_y  \
0             260              15             -53           51           54   
1             259              13             -56           16           22   
2             262               6             -57          -20           78   
3             255               8             -54            1          -11   
4             258               9             -58          -10           64   

   gyroscope_z  

/var/folders/q4/b_qkk2qx16zdglt42g7c0dlr0000gn/T/ipykernel_86754/4023679038.py:8: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframe = pd.read_json(raw_data)


In [11]:
dataframe['time'] = pd.to_datetime(dataframe['time'], format='mixed')
dataframe['time_second'] = dataframe.time.map(lambda x: pd.Timestamp(x).floor(freq='S'))
dataframe['time'] = dataframe.time.map(pd.Timestamp.timestamp)

grouped = dataframe.groupby([dataframe.trip_id, dataframe.time_second])  # grouped.get_group(1)
x = []
id_list, time_list, lat_list, lon_list = [], [], [], []

for i, (trip_seconds, table) in enumerate(grouped):
    time_list.append(trip_seconds[0])
    id_list.append(trip_seconds[1])
    lat_list.append(table['latitude'].mean())
    lon_list.append(table['longitude'].mean())

    train_input = table.drop(columns=['terrain', 'trip_id', 'crash', 'time_second', 'latitude', 'longitude'])
    if len(train_input.columns) != config.n_training_cols:
        raise Exception("Wrong number of input columns (should be)", config.n_training_cols,
                        ", please check data.\n"
                        "But input was: ", train_input)

    train_input = train_input.to_numpy()

    input_length = len(train_input)
    if input_length > config.batch_size:
        train_input = train_input[:config.batch_size]
    else:
        n_missing_rows = config.batch_size - len(train_input)
        for _ in range(n_missing_rows):
            fake_array = [1] * config.n_training_cols
            train_input = np.append(train_input, fake_array)

    x.append(train_input)



y = []


0
1970-01-01 00:00:00
207
1970-01-01 00:00:00
208
1970-01-01 00:00:00
209
1970-01-01 00:00:00
210
1970-01-01 00:00:00
211
1970-01-01 00:00:00
212
1970-01-01 00:00:00
213
1970-01-01 00:00:00
214
1970-01-01 00:00:00
215
1970-01-01 00:00:00
216
1970-01-01 00:00:00
217
1970-01-01 00:00:00
218
1970-01-01 00:00:00
219
1970-01-01 00:00:00
220
1970-01-01 00:00:00
221
1970-01-01 00:00:00
222
1970-01-01 00:00:00
223
1970-01-01 00:00:00
224
1970-01-01 00:00:00
225
1970-01-01 00:00:00
226
1970-01-01 00:00:00
227
1970-01-01 00:00:00
228
1970-01-01 00:00:00
229
1970-01-01 00:00:00
230
1970-01-01 00:00:00
231
1970-01-01 00:00:00
232
1970-01-01 00:00:00
233
1970-01-01 00:00:00
234
1970-01-01 00:00:00
235
1970-01-01 00:00:00
245
1970-01-01 00:00:00
255
1970-01-01 00:00:00
265
1970-01-01 00:00:00
275
1970-01-01 00:00:00
285
1970-01-01 00:00:00
295
1970-01-01 00:00:00
305
1970-01-01 00:00:00
306
1970-01-01 00:00:00
307
1970-01-01 00:00:00
308
1970-01-01 00:00:00
309
1970-01-01 00:00:00
310
1970-01-01 00: